In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MaxAbsScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [91]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

### Data Import

In [2]:
train = pd.read_csv("./data/train_20171226.csv")
test = pd.read_csv("./data/yancheng_testB_20180224.csv")
print(train.shape,test.shape)

(20157, 32) (140, 3)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.shape

(20157, 32)

In [4]:
test.shape

(140, 3)

In [8]:
train.head()

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,...,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201609,289403,94,12,2,1,1,1,6,MT,...,170,4440,1833,1545,1695,1320,5,2700,1556,1562
1,201609,745137,435,637,3,2,1,2,6,DCT,...,159,4534,1823,1483,1711,1336,5,2648,1553,1544
2,201609,714860,180,831,3,2,2,3,6,AT,...,176,4720,1815,1465,1860,1459,5,2770,1579,1589
3,201609,175962,40,750,3,2,1,4,6,AT,...,155,4475,1706,1469,1625,1145,5,2603,1460,1500
4,201609,270690,19,98,2,3,3,1,5,MT,...,146.5,4415,1685,1850,1825,1236,5,2720,1420,1440


In [24]:
print(list(train))

['sale_date', 'class_id', 'sale_quantity', 'brand_id', 'compartment', 'type_id', 'level_id', 'department_id', 'TR', 'gearbox_type', 'displacement', 'if_charging', 'price_level', 'price', 'driven_type_id', 'fuel_type_id', 'newenergy_type_id', 'emission_standards_id', 'if_MPV_id', 'if_luxurious_id', 'power', 'cylinder_number', 'engine_torque', 'car_length', 'car_width', 'car_height', 'total_quality', 'equipment_quality', 'rated_passenger', 'wheelbase', 'front_track', 'rear_track', 'sale_month', 'sale_year']


In [25]:
print(train.isna().sum())

sale_date                0
class_id                 0
sale_quantity            0
brand_id                 0
compartment              0
type_id                  0
level_id                 0
department_id            0
TR                       0
gearbox_type             0
displacement             0
if_charging              0
price_level              0
price                    0
driven_type_id           0
fuel_type_id             0
newenergy_type_id        0
emission_standards_id    0
if_MPV_id                0
if_luxurious_id          0
power                    0
cylinder_number          0
engine_torque            0
car_length               0
car_width                0
car_height               0
total_quality            0
equipment_quality        0
rated_passenger          0
wheelbase                0
front_track              0
rear_track               0
sale_month               0
sale_year                0
dtype: int64


In [26]:
print(train.dtypes)

sale_date                  int64
class_id                   int64
sale_quantity              int64
brand_id                   int64
compartment                int64
type_id                    int64
level_id                  object
department_id              int64
TR                        object
gearbox_type              object
displacement             float64
if_charging               object
price_level               object
price                     object
driven_type_id             int64
fuel_type_id              object
newenergy_type_id          int64
emission_standards_id      int64
if_MPV_id                  int64
if_luxurious_id            int64
power                     object
cylinder_number            int64
engine_torque             object
car_length                 int64
car_width                  int64
car_height                 int64
total_quality              int64
equipment_quality          int64
rated_passenger           object
wheelbase                  int64
front_trac

In [36]:
train = train.sort_values(by=['sale_date','class_id']).reset_index(drop=True)

In [40]:
train[:2]

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,price_level,price,driven_type_id,fuel_type_id,newenergy_type_id,emission_standards_id,if_MPV_id,if_luxurious_id,power,cylinder_number,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_month,sale_year
0,201201,125403,49,761,2,3,2,2,6,AT,2.4,L,35-50W,-,1,1,1,3,2,1,123,4,225,5256,1878,1772,2470,1840,7,3088,1593,1601,1,12
1,201201,125403,16,761,2,3,2,2,6,AT,2.4,L,35-50W,-,1,1,1,3,2,1,123,4,225,5213,1847,1750,2380,1840,7,3088,1593,1601,1,12


### Data Exploration

In [22]:
train['sale_month'] = train['sale_date']%100
train['sale_year'] = train['sale_date']//100%100
test['sale_month'] = test['predict_date']%100
test['sale_year'] = test['predict_date']//100%100

In [46]:
# only include the maximum sale quantity rows for each class_id & month, and put the total sales in there
# so the dataframe merge_train have unique class_id * month rows
merge_train = pd.DataFrame()
all_class_id = train['class_id'].unique().tolist()

for class_id in all_class_id:
    all_month = train.loc[train['class_id']== class_id, 'sale_date'].unique().tolist()
    for month in all_month:
        max_val  = train.loc[(train['class_id']==class_id) & (train['sale_date']==month), 'sale_quantity'].max()
        sale_sum = train.loc[(train['class_id']==class_id) & (train['sale_date']==month), 'sale_quantity'].sum()
        feat_val = train.loc[(train['class_id']==class_id) & (train['sale_date']==month) & (train['sale_quantity']==max_val), :]
        feat_val['sale_quantity'] = sale_sum
        merge_train = pd.concat([merge_train, feat_val[0:1]])
        
merge_train = merge_train.sort_values(by=['sale_date'], ascending=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [47]:
merge_train.shape

(5587, 34)

In [48]:
merge_train = merge_train[['sale_date', 'class_id', 'sale_quantity', 'sale_month', 'sale_year']]
cols = ['sale_date', 'class_id', 'sale_quantity', 'sale_month', 'sale_year']
############### Remove Nov ##################
merge_train = merge_train.loc[merge_train['sale_month']!=11, :]
test = test.loc[:, cols]
test['sale_date'] = test['sale_date'].fillna(201712)
test['sale_date'] = test['sale_date'].astype(int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [68]:
# looking for class_id that has less than 14 training months
months_data_per_class = merge_train[['class_id','sale_quantity']].groupby(['class_id']).count().reset_index()

In [69]:
print('Number of classes with less than 14 months records :', 
      months_data_per_class.loc[months_data_per_class['sale_quantity'] < 14, 'class_id'].shape[0])

Number of classes with less than 14 months records : 34


In [70]:
print(merge_train.shape, test.shape)
class_12 = months_data_per_class.loc[months_data_per_class['sale_quantity']<14, 'class_id']
class_12 = class_12.tolist()
all_data = pd.concat((merge_train, test)).reset_index(drop=True)

(5195, 5) (140, 5)


In [71]:
all_data.head()

,sale_date,class_id,sale_quantity,sale_month,sale_year
0,201201,125403,90.0,1,12
1,201201,628041,186.0,1,12
2,201201,651347,54.0,1,12
3,201201,290854,632.0,1,12
4,201201,682651,373.0,1,12


In [74]:
### remove all the class_ids with less than 14 months records
all_data = all_data.loc[~all_data['class_id'].isin(class_12), :]

In [75]:
all_data.shape

(5034, 5)

In [84]:
print("Before removing claass_id, we have", 
      len(all_class_id), "classes")

Before removing claass_id, we have 140 classes


In [83]:
print("After removing class_id with less than 14 months record, we have", 
      len(np.unique(all_data['class_id'])), "classes left")

After removing class_id with less than 14 months record, we have 106 classes left


In [76]:
np.unique(all_data['sale_date'])

array([201201, 201202, 201203, 201204, 201205, 201206, 201207, 201208,
       201209, 201210, 201212, 201301, 201302, 201303, 201304, 201305,
       201306, 201307, 201308, 201309, 201310, 201312, 201401, 201402,
       201403, 201404, 201405, 201406, 201407, 201408, 201409, 201410,
       201412, 201501, 201502, 201503, 201504, 201505, 201506, 201507,
       201508, 201509, 201510, 201512, 201601, 201602, 201603, 201604,
       201605, 201606, 201607, 201608, 201609, 201610, 201612, 201701,
       201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709,
       201710, 201712])

In [85]:
all_data.groupby(['class_id'])['sale_date'].apply(lambda x : len(np.unique(x)))

class_id
103507    31
124140    40
125403    66
136916    66
175962    56
178529    66
186250    21
194450    65
198427    50
206765    27
209945    66
219195    37
221795    52
245609    19
248352    66
250658    20
265980    65
270690    33
281301    66
281792    39
289386    39
289403    18
290854    66
291086    66
291514    57
          ..
714860    36
732758    62
735971    66
736094    22
739296    38
741152    35
745137    62
750340    27
786351    58
810398    26
819061    66
851857    19
854079    49
871642    51
883691    66
905745    56
914348    66
923841    64
924154    33
950264    16
953842    21
961362    66
961962    66
963845    35
978089    37
Name: sale_date, Length: 106, dtype: int64

In [77]:
np.unique(merge_train['sale_date'])

array([201201, 201202, 201203, 201204, 201205, 201206, 201207, 201208,
       201209, 201210, 201212, 201301, 201302, 201303, 201304, 201305,
       201306, 201307, 201308, 201309, 201310, 201312, 201401, 201402,
       201403, 201404, 201405, 201406, 201407, 201408, 201409, 201410,
       201412, 201501, 201502, 201503, 201504, 201505, 201506, 201507,
       201508, 201509, 201510, 201512, 201601, 201602, 201603, 201604,
       201605, 201606, 201607, 201608, 201609, 201610, 201612, 201701,
       201702, 201703, 201704, 201705, 201706, 201707, 201708, 201709,
       201710])

In [78]:
np.unique(test['sale_date'])

array([201712])

In [86]:
all_data

,sale_date,class_id,sale_quantity,sale_month,sale_year
0,201201,125403,90.0,1,12
1,201201,628041,186.0,1,12
2,201201,651347,54.0,1,12
3,201201,290854,632.0,1,12
4,201201,682651,373.0,1,12
5,201201,713651,192.0,1,12
6,201201,714152,298.0,1,12
7,201201,281301,965.0,1,12
8,201201,735971,877.0,1,12
9,201201,819061,640.0,1,12


In [79]:
def rolling(df,roll,window):
    last=roll(df, window=window).tolist()
    last=[np.NaN]+last
    last.pop()
    return last

In [ ]:
new = pd.DataFrame()
for class_id in all_class_id:
    df = all_data.loc[all_data['class_id'] == class_id, :]
    df['last_1_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=1)
    df['last_2_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=2)
    df['last_3_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=3)
    df['last_4_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=4)
    df['last_5_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=5)
    df['last_6_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=6)
    df['last_7_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=7)
    df['last_8_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=8)
    df['last_9_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=9)
    df['last_10_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=10)
    df['last_11_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=11)
    df['last_12_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=12)
    df['last_13_sum'] = rolling(df['sale_quantity'], pd.rolling_sum, window=13)
    
    df['last_5_mean'] = rolling(df['sale_quantity'], pd.rolling_mean, window=5)
    df['last_5_std'] = rolling(df['sale_quantity'], pd.rolling_std, window=5)
    
    df['move_1'] = df['last_1_sum']
    df['move_2'] = df['last_2_sum'] - df['last_1_sum']
    df['move_3'] = df['last_3_sum'] - df['last_2_sum']
    df['move_4'] = df['last_4_sum'] - df['last_3_sum']
    df['move_5'] = df['last_5_sum'] - df['last_4_sum']
    
    df['move_9'] = df['last_9_sum'] - df['last_8_sum']
    
    df['move_11'] = df['last_11_sum'] - df['last_10_sum']
    df['move_12'] = df['last_12_sum'] - df['last_11_sum']
    df['move_13'] = df['last_13_sum'] - df['last_12_sum']
    
    
    df['diff_12'] = df['move_11'] - df['move_12']
    new = pd.concat([new, df])

In [87]:
temp = all_data.loc[all_data['class_id'] == 125403, :]

In [98]:
temp

,sale_date,class_id,sale_quantity,sale_month,sale_year
0,201201,125403,90.0,1,12
62,201202,125403,156.0,2,12
119,201203,125403,91.0,3,12
158,201204,125403,74.0,4,12
199,201205,125403,130.0,5,12
249,201206,125403,164.0,6,12
296,201207,125403,188.0,7,12
326,201208,125403,131.0,8,12
363,201209,125403,124.0,9,12
440,201210,125403,107.0,10,12


In [100]:
temp['sale_quantity'].rolling(3).sum()

0          NaN
62         NaN
119      337.0
158      321.0
199      295.0
249      368.0
296      482.0
326      483.0
363      443.0
440      362.0
454      354.0
531      468.0
553      451.0
622      484.0
706      385.0
739      525.0
813      500.0
845      607.0
903      532.0
981      573.0
1045     441.0
1069     385.0
1152     377.0
1175     352.0
1292     409.0
1325     475.0
1385     451.0
1434     401.0
1490     393.0
1556     475.0
1670     508.0
1751     384.0
1786     401.0
1892     409.0
1941     426.0
2019     402.0
2104     525.0
2208     500.0
2261     524.0
2366     467.0
2408     590.0
2541     623.0
2583     525.0
2736     525.0
2829     484.0
2904     484.0
2964     468.0
3103     673.0
3166     714.0
3318     649.0
3363     460.0
3542     501.0
3557     616.0
3772     543.0
3829     528.0
3991     453.0
4024     485.0
4163     663.0
4372     818.0
4484     966.0
4529     949.0
4727    1040.0
4897    1072.0
4948    1057.0
5076     795.0
5229       NaN
Name: sale

In [95]:
temp.groupby(['sale_year'])['sale_month'].count()

sale_year
12    11
13    11
14    11
15    11
16    11
17    11
Name: sale_month, dtype: int64

In [102]:
pd.rolling_sum(temp, window=1)

AttributeError: module 'pandas' has no attribute 'rolling_sum'

In [103]:
all_data.head(3)

,sale_date,class_id,sale_quantity,sale_month,sale_year
0,201201,125403,90.0,1,12
1,201201,628041,186.0,1,12
2,201201,651347,54.0,1,12


In [104]:
all_data.groupby(['class_id'])['sale_date'].count()

class_id
103507    31
124140    40
125403    66
136916    66
175962    56
178529    66
186250    21
194450    65
198427    50
206765    27
209945    66
219195    37
221795    52
245609    19
248352    66
250658    20
265980    65
270690    33
281301    66
281792    39
289386    39
289403    18
290854    66
291086    66
291514    57
304458    66
308913    49
321683    66
347384    57
348641    25
349023    66
350259    33
354068    66
356986    47
360648    44
378356    20
379876    66
384924    66
392593    15
395234    58
401052    29
409854    66
412240    18
416749    32
425432    66
436105    45
437598    28
453752    65
468392    66
472603    66
          ..
526401    20
527809    66
540712    20
559132    66
560265    46
576298    66
580470    66
580634    65
587678    34
591790    37
597927    66
603972    66
614059    66
619401    16
621073    48
628041    66
649213    36
651347    66
651782    53
654134    27
671482    51
682651    66
692703    29
713651    66
714152    66
714